## Question (a)

In [23]:
import pandas as pd
import numpy as np
from sklearn import cross_validation, naive_bayes , metrics

df = pd.read_json('./all/train.json')

## Question (b)

In [2]:
num_obs = df.shape[0]
num_obs

39774

Total number of the samples is: 39774

In [3]:
cuisine_array=df.cuisine.unique()
print cuisine_array
print cuisine_array.size

[u'greek' u'southern_us' u'filipino' u'indian' u'jamaican' u'spanish'
 u'italian' u'mexican' u'chinese' u'british' u'thai' u'vietnamese'
 u'cajun_creole' u'brazilian' u'french' u'japanese' u'irish' u'korean'
 u'moroccan' u'russian']
20


There are totally 20 types of cuisines.

In [4]:
ingredients_collection=df['ingredients'].values
# print ingredients_collection

ingredients = set()
for i in ingredients_collection:
    ingredients = ingredients.union(i)
    
ing_list = list(ingredients)

In [5]:
len(ing_list)

6714

There are totally 6714 unique ingredients.

## Question (c)

In [9]:
%%time
print 'start'
featureList = []

for i in range(num_obs): #len(trainData)
    feature = [0] * len(ing_list)
    sub_list = ingredients_collection[i]
    for j in range(len(ing_list)):
        if ing_list[j] in sub_list:
            #print ("yes",ing_list[j],j)
            feature[j] = 1
    featureList.append(feature)

featureMatrix = np.matrix(featureList)
print featureMatrix.shape
print 'end'

start
(39774L, 6714L)
end
Wall time: 3min 27s


## Question (d)

In [10]:
fold1=list(cross_validation.KFold(num_obs, n_folds=3))[0]
fold2=list(cross_validation.KFold(num_obs, n_folds=3))[1]
fold3=list(cross_validation.KFold(num_obs, n_folds=3))[2]

In [14]:
def cross_validation_Naive_Bayes_evaluate(dataSet,labels,fold_indices,NB):
    train_indices, test_indices = fold_indices
    X_train = dataSet[train_indices]
    Y_train = labels[train_indices]
    X_test = dataSet[test_indices]
    Y_test = labels[test_indices]
    
    if(NB == 'Bernoulli'):
        clf = naive_bayes.BernoulliNB()
    elif(NB == 'Gaussian'):
        clf = naive_bayes.GaussianNB()
    clf.fit(X_train, Y_train)
    # BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)
    Y_estimate = clf.predict(X_test)
    return (Y_estimate,Y_test)

In [24]:
def accuracy(Y_predict,Y_true):
    return metrics.accuracy_score(Y_true, Y_predict)

In [26]:
print 'Bernoulli'
(Y_predict, Y_true) = cross_validation_Naive_Bayes_evaluate(featureMatrix,df.cuisine,fold1,'Bernoulli')
acc1 = accuracy(Y_predict,Y_true)
print acc1
(Y_predict, Y_true) = cross_validation_Naive_Bayes_evaluate(featureMatrix,df.cuisine,fold2,'Bernoulli')
acc2 = accuracy(Y_predict,Y_true)
print acc2
(Y_predict, Y_true) = cross_validation_Naive_Bayes_evaluate(featureMatrix,df.cuisine,fold3,'Bernoulli')
acc3 = accuracy(Y_predict,Y_true)
print acc3

Bernoulli
0.684190677326897
0.6795142555438226
0.6869060190073918


In [27]:
print 'Gaussian'
(Y_predict, Y_true) = cross_validation_Naive_Bayes_evaluate(featureMatrix,df.cuisine,fold1,'Gaussian')
acc1 = accuracy(Y_predict,Y_true)
print acc1
(Y_predict, Y_true) = cross_validation_Naive_Bayes_evaluate(featureMatrix,df.cuisine,fold2,'Gaussian')
acc2 = accuracy(Y_predict,Y_true)
print acc2
(Y_predict, Y_true) = cross_validation_Naive_Bayes_evaluate(featureMatrix,df.cuisine,fold3,'Gaussian')
acc3 = accuracy(Y_predict,Y_true)
print acc3

Gaussian
0.37901644290239855
0.3829386031075577
0.37758334590435966


## Question (e)

Bernoulli prior is much better than the Gaussian prior in terms of cross-validation accuracy.
The reason is that the vector of each feature is a binary vector. Gaussian prior works better when the values of feature are
continuous values.

## Question (f)

In [34]:
from sklearn.linear_model import LogisticRegression

In [35]:
def cross_validation_Logistic_Regression_evaluate(dataSet,labels,fold_indices):
    train_indices, test_indices = fold_indices
    X_train = dataSet[train_indices]
    Y_train = labels[train_indices]
    X_test = dataSet[test_indices]
    Y_test = labels[test_indices]
        
    clf = LogisticRegression(multi_class='ovr')
    clf.fit(X_train, Y_train)
    Y_estimate = clf.predict(X_test)
    return (Y_estimate,Y_test)

In [36]:
print 'Logistic Regression'
(Y_predict, Y_true) = cross_validation_Logistic_Regression_evaluate(featureMatrix,df.cuisine,fold1)
acc1 = accuracy(Y_predict,Y_true)
print acc1
(Y_predict, Y_true) = cross_validation_Logistic_Regression_evaluate(featureMatrix,df.cuisine,fold2)
acc2 = accuracy(Y_predict,Y_true)
print acc2
(Y_predict, Y_true) = cross_validation_Logistic_Regression_evaluate(featureMatrix,df.cuisine,fold3)
acc3 = accuracy(Y_predict,Y_true)
print acc3

Logistic Regression
0.7758334590435964
0.7721375773118118
0.7786996530396741


## Question (g)

The best-performed classiﬁer is logistic regression classifier.

In [30]:
df2 = pd.read_json('./all/test.json')

start
(59662L, 6714L)
end


In [51]:
df2.shape

(9944, 2)

In [58]:
print 'start'
featureList2 = []

for i in range(df2.shape[0]): #len(trainData)
    feature = [0] * len(ing_list)
    sub_list = (df2['ingredients'].values)[i]
    for j in range(len(ing_list)):
        if ing_list[j] in sub_list:
            feature[j] = 1
    featureList2.append(feature)

featureMatrix2 = np.matrix(featureList2)
print featureMatrix2.shape
print 'end'

start
(9944L, 6714L)
end


In [60]:
clf = LogisticRegression(multi_class='ovr')
clf.fit(featureMatrix, df.cuisine)
Y_estimate = clf.predict(featureMatrix2)

In [61]:
# Output the solution
import csv

file_name = "submission.csv"
with open(file_name, mode='wb') as submission_file:
    fieldnames = ['id', 'cuisine']
    writer = csv.DictWriter(submission_file, fieldnames=fieldnames)
    
    writer.writerow({'id': 'id', 'cuisine': 'cuisine'})
    for i in range(len(Y_estimate)):
        writer.writerow({'id': df2['id'][i], 'cuisine': Y_estimate[i]})
    
print "done"

done


In [62]:
Y_estimate

array([u'british', u'southern_us', u'italian', ..., u'italian',
       u'southern_us', u'mexican'], dtype=object)